<a href="https://colab.research.google.com/github/RaulRU30/UPSLP-Chatbot/blob/main/IA2-Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Poli-Chatbot

##### Autor: Raúl Reyes Urbina


---


Proyecto para el primer parcial de la materia de Inteligencia Artificial II, diseñado para responder preguntas frecuentes relacionadas con la UPSLP.


---



## Ejemplos de Preguntas por Categoría

La siguiente tabla muestra una pregunta de ejemplo por cada categoría del chatbot. Estas preguntas están diseñadas para coincidir con los patrones regex adaptados a texto *stemmizado*.

| Pregunta                 | Ejemplo                                                  |
|--------------------------|----------------------------------------------------------------------|
| Inscripciones semestre | ¿Cuándo son las inscripciones para otoño 2025?                      |
| Ficha de pago          | ¿Cómo obtengo mi ficha de pago?                                     |
| Pago cursos de verano  | ¿Cómo pago los cursos de verano?                                    |
| Fechas cursos de verano| ¿Cuándo son los cursos de verano?                                   |
| Dirección universidad  | ¿Cuál es la dirección de la UPSLP?                                  |
| Contacto universidad   | ¿A qué correo puedo escribir?                                       |
| Días festivos          | ¿Qué días no hay clases?                                            |
| Rector universidad     | ¿Quién es el rector de la UPSLP?                                    |
| Fundación universidad   | ¿Cuándo se fundó la UPSLP?                                          |
| Oferta académica       | ¿Qué carreras ofrece la universidad?                               |
| Admisión               | ¿Cómo me puedo inscribir a la uni?                                 |
| Calendario escolar     | ¿Dónde consulto el calendario escolar?                             |
| Ayuda general          | ¿Me puedes ayudar?                                                 |


In [2]:
import json
import re
import sys
import unicodedata
from typing import Tuple

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Descarga de informacion de NLTK
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt")

try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# Clase encargada de procesar el texto para que pueda ser interpretado por las
# expresiones regulares, aplica normalizacion, filtrado de stop workds y
# stemmizacion

class TextProcessor:

    def __init__(self):
        language = "spanish"
        self.stemmer = SnowballStemmer(language)
        self.stop_words = set(stopwords.words(language))

    def normalize_text(self, text: str) -> str:
        text = unicodedata.normalize("NFD", text)
        text = "".join(char for char in text if unicodedata.category(char) != "Mn")
        return text.lower()

    def remove_stopwords(self, text: str) -> str:
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in self.stop_words]
        return " ".join(filtered_words)

    def apply_stemming(self, text: str) -> str:
        words = text.split()
        stemmed_words = [self.stemmer.stem(word) for word in words]
        return " ".join(stemmed_words)

    def process_text(self, text: str) -> str:
        text = self.normalize_text(text)
        text = self.remove_stopwords(text)
        text = self.apply_stemming(text)
        print("Texto procesado: " + text)
        return text

In [6]:
# Clase encargada de manejar la lectura de la KB en json, procesar el input del
# usuario y hacer match con alguna expresion regular.
class RuleBasedChatbot:

    def __init__(self, rules_file):
        self.text_processor = TextProcessor()
        self.rules = []
        self.default_response = (
            "Lo siento, no entiendo tu pregunta. ¿Podrías reformularla?"
        )

        self.load_rules_from_file(rules_file)

    def load_rules_from_file(self, file_path: str) -> None:
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)
                self.rules = data.get("regex_chatbot", [])
        except FileNotFoundError:
            print(f"Archivo {file_path} no encontrado.")
            sys.exit(1)
        except json.JSONDecodeError:
            print(f"Error al leer el archivo {file_path}. Formato JSON inválido.")
            sys.exit(1)

    def get_response(self, user_input: str) -> Tuple[str, bool]:
        processed_input = self.text_processor.process_text(user_input)
        original_input = self.text_processor.normalize_text(user_input)

        for rule in self.rules:
            pattern = rule["pattern"]

            if re.search(pattern, processed_input, re.IGNORECASE):
                return rule["respuesta"], True

            if re.search(pattern, original_input, re.IGNORECASE):
                return rule["respuesta"], True

        return self.default_response, False

    def set_default_response(self, response: str) -> None:
        self.default_response = response

In [7]:
# Clase manager del chatbot, se encarga de mantener el ciclo de ejecucion corriendo
# asi como manejar el input/output del usuario
class ChatbotManager:

    def __init__(self, chatbot: RuleBasedChatbot):
        self.chatbot = chatbot
        self.conversation_history = []
        self.running = False

    def start_conversation(self) -> None:
        print("=" * 60)
        print("POLI CHATBOT")
        print("=" * 60)
        print("Escribe 'salir', 'exit' o 'quit' para terminar.")
        print("Escribe 'ayuda' para ver comandos disponibles.")
        print("-" * 60)

        self.running = True

        while self.running:
            try:
                user_input = input("\nTú: ").strip()

                if not user_input:
                    continue

                if self._handle_commands(user_input):
                    continue

                response, found = self.chatbot.get_response(user_input)

                status_icon = "✅" if found else "❓"
                print(f"\n🤖 Bot {status_icon}: {response}")

            except KeyboardInterrupt:
                print("\n\n👋 ¡Hasta luego!")
                break
            except Exception as e:
                print(f"\n❌ Error: {e}")

    def _handle_commands(self, user_input: str) -> bool:
        command = user_input.lower()

        if command in ["salir", "exit", "quit"]:
            print("\n ¡Gracias por usar el chatbot! Hasta luego.")
            self.running = False
            return True

        elif command == "ayuda":
            self._show_help()
            return True

        return False

    def _show_help(self) -> None:
        """Muestra ayuda disponible"""
        print("\n COMANDOS DISPONIBLES:")
        print("• 'ayuda' - Muestra este mensaje")
        print("• 'salir' - Termina la conversacion")
        print("\n EJEMPLOS DE PREGUNTAS:")
        print("• ¿Cuándo son las inscripciones para otoño 2025?")
        print("• ¿Cómo obtengo mi ficha de pago?")
        print("• ¿Cuándo son los cursos de verano?")
        print("• ¿Cuál es la dirección de la UPSLP?")
        print("• ¿A qué correo puedo escribir?")
        print("• ¿Quién es el rector de la universidad?")
        print("• ¿Cuáles son los días festivos?")
        print("• ¿Qué carreras ofrece la universidad?")


In [9]:
# Clase main donde se inicializa el chatbot

def main():
    file_path = "/content/drive/MyDrive/Colab Notebooks/base_conocimiento.json"
    github_file_path = "https://github.com/RaulRU30/UPSLP-Chatbot/blob/main/base_conocimiento.json"
    chatbot = RuleBasedChatbot(github_file_path)

    manager = ChatbotManager(chatbot)
    manager.start_conversation()


if __name__ == "__main__":
    main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Archivo https://github.com/RaulRU30/UPSLP-Chatbot/blob/main/base_conocimiento.json no encontrado.
Traceback (most recent call last):
  File "/tmp/ipython-input-6-1742902639.py", line 16, in load_rules_from_file
    with open(file_path, "r", encoding="utf-8") as file:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'https://github.com/RaulRU30/UPSLP-Chatbot/blob/main/base_conocimiento.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-9-1751833291.py", line 13, in <cell line: 0>
    main()
  File "/tmp/ipython-input-9-1751833291.py", line 6, in main
    chatbot = RuleBasedChatbot(github_file_path)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-6-1742902639.py",

TypeError: object of type 'NoneType' has no len()